In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# imports
import matplotlib.pyplot as plt
from scipy import signal
import seaborn as sns
import glob
import datetime
import os
from tempfile import TemporaryFile

In [ ]:
# Data
Diractory = '../input/predict-volcanic-eruptions-ingv-oe'
train = pd.read_csv(os.path.join(Diractory, 'train.csv'))
test_submission = pd.read_csv(os.path.join(Diractory, 'sample_submission.csv'))

train

In [ ]:
# convert time to eruption to days, hours, minutes
#train['h:m:s'] = (train['time_to_eruption']
#                  .apply(lambda x:datetime.timedelta(seconds = x/100)))
#display train
train

In [ ]:
# Getting sample ids
sample_ids = train['segment_id'].values

sample_ids


In [ ]:
# Looking at every sigment_id file and cleaning the NaN values by replacing it with the mean of every sample.
# Need to extend to every segment_id
#segment_id = '1136037770'
#segments = []
# for i, segment_id in enumerate(sample_ids):
#temp = pd.read_csv(os.path.join(Diractory, f'train/{segment_id}.csv')).T
#temp_mean = temp.mean()
#segments.append(temp.fillna(temp_mean).T)

#segments

In [ ]:

# segment_id = '1136037770'
sensor_1 = np.zeros((4431,60001))
print('preparing sensor_1')
for i, segment_id in enumerate(sample_ids):
    temp = pd.read_csv(os.path.join(Diractory, f'train/{segment_id}.csv'))
    sensor_1[i,:] = temp['sensor_10']
    if (i % 200 == 0):
        print(i)
    #print(i)

print('done')
# temp_mean = temp.mean()
# segments.append(temp.fillna(temp_mean).T)
#sensors[0,:] = temp['sensor_1']
#sensors[0,:]
#segments

In [ ]:
# time to eruption array
time_to_eruption = train['time_to_eruption'].values
print(time_to_eruption)
print(time_to_eruption.shape)
#print()
#print(time_to_eruption.size)

In [ ]:
# patial and empty data analysis for sensor_1
count_empty = 0
count_partial = 0
i = 0
segments_id_to_remove = []
for segment_id in sensor_1:
    count_Nan = np.count_nonzero(np.isnan(segment_id))
    # print(count_Nan)
    
    if (count_Nan > 0) & (count_Nan < 60001):
        print(count_Nan)
        segments_id_to_remove.append(i)
        
        count_partial = count_partial + 1
    if count_Nan == 60001:
        print(count_Nan)
        count_empty = count_empty + 1
        segments_id_to_remove.append(i)
        
    i = i + 1
print('Number of empty files for sensor_1: ',count_empty)
print('Number of partial files for sensor_1:',count_partial)

#we chose to delete all partial and empty segments for now,
#as well as their time_to_eruption

# print('Need to remove indices' + str(segments_id_to_remove))
print()
sensor_1_clean = np.delete(sensor_1,segments_id_to_remove,axis=0)
print(sensor_1_clean)
print("sensor 1 clean shape: "+str(sensor_1_clean.shape) + str(type(sensor_1_clean)))
print()
#print(train)
#remove title
#time_to_eruption_clean1 = np.delete(time_to_eruption,0,axis=0)
#print(time_to_eruption_clean1.shape)
#print(train.to_numpy())
times_and_ids_clean = np.delete(train.to_numpy(),segments_id_to_remove,axis=0)
print(times_and_ids_clean)
#print("time to eruption clean shape: " +str(time_to_eruption_clean.shape))
# segments_ids_clean = np.delete(train['segment_id'].values,segments_id_to_remove,axis=0)
# print("segment_ids clean shape: " +str(segments_ids_clean.shape))

#times_and_ids_clean = [segments_ids_clean,time_to_eruption_clean]
print("times and ids shape: "+str(times_and_ids_clean.shape) + str(type(times_and_ids_clean)))


In [ ]:
# checking data is clean
count_empty = 0
count_partial = 0
i = 0
segments_id_to_remove = []
for segment_id in sensor_1_clean:
    count_Nan = np.count_nonzero(np.isnan(segment_id))
    # print(count_Nan)
    
    if (count_Nan > 0) & (count_Nan < 60001):
        print(count_Nan)
        segments_id_to_remove.append(i)
        
        count_partial = count_partial + 1
    if count_Nan == 60001:
        print(count_Nan)
        count_empty = count_empty + 1
        segments_id_to_remove.append(i)
        
    i = i + 1
print('Number of empty files for sensor_1: ',count_empty)
print('Number of partial files for sensor_1:',count_partial)

In [ ]:
# save array to csv
# os.path.join('../input/predict-volcanic-eruptions-ingv-oe')
# np.savetxt('sensor_1_clean.csv', sensor_1_clean, delimiter=',')

In [ ]:
sensor_1_clean.shape
sensor_1_clean.shape[0]

In [ ]:
# psd of sensor_1 cleaned

PSD_sensor_1 = []

for i in range(sensor_1_clean.shape[0]):
    #col_mean = df_example[col].mean()
    #df_example[col].fillna(col_mean, inplace=True)
    freq, psd = signal.welch(sensor_1_clean[i],100)
    plt.loglog(freq,psd)
    PSD_sensor_1.append(psd)
    if i % 1000 == 0:
        print(i)
print('PSD done')
PSD_sensor_1 = np.asarray(PSD_sensor_1)
print("Shape of PSD: "+ str(PSD_sensor_1.shape) + str(type(PSD_sensor_1)))

In [ ]:
# save psd and sensor_x_clean, time_to eruption to numpy file
os.path.join('../output/kaggle/working/sensors_data')
np.save('sensor_10_time_to_eruption', times_and_ids_clean)
os.path.join('../output/kaggle/working/sensors_data')
np.save('sensor_10_clean', sensor_1_clean)
os.path.join('../output/kaggle/working/psd')
np.save('sensor_10_psd', PSD_sensor_1)
os.path.join('../output/kaggle/working/freq')
np.save('freq_10', freq)